In [ ]:
#Normal Imports/settings
import os
import math
import numpy as np
import pandas as pd
#import seaborn as sns
from os import listdir
import matplotlib.pyplot as plt
from os.path import isfile, join
pd.set_option('display.max_rows', 10)
from IPython.display import HTML, display
pd.set_option('display.max_columns', None)

#Sklearn imports
from sklearn import tree
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

#Provided code imports 
import daml
from daml import plot, utilities, optimisation

#Self-written additional code import
from Utility import *

#### Notes
   - The seaborn module was used throughout this notebook, however i am unsure as to if it is installed on the CP lab machines and so all code referring to it is commented out. In place of the methods used form this library, the 'daml' folder of pre-written code has been brought over and used instead.
   - Throughout this notebook, the Utility file imported will be used for some of the data processing and handling. This is to make this notebook less bogged down with random functions and hopefully easier to follow.
   - The file should have been zipped with the project 
   - The functions themselves have docstrings and so can have their use quickly identified from the notebook if the function name wasn't suffice
   - As the data is hand generated it is always aimed that the classes of particles are balanced for classification, with the small exception of some unphysical data removed from time to time. This is a fairly rare occurence and so in this case the idea of balanced classes still holds perfectly well. As the classes are balanced, the simple accuracy score metric is suitable for use.

# First Investigation - Detector 1
 - Detector one is constructed with the following layer:
     - A front shield made of lead 
     - 5 layers of liquid argon as the central detecting layers
     - a final large lead absorbing plate at the back of the experiment

In [ ]:
display(HTML("<table><tr><td><img src='pics/detec1_1.png'></td><td><img src='pics/detec1_2.png'></td></tr></table>"))

### Import Data
 - For detector one, electrons, muons and photons were  the particles simulated with.
 - The first data set for each is 5000 simulated events at 100 MeV with no magnetic field
 - the second data set for each is 1000 events of increasing energy, starting at 10 MeV and adding an additional 10 MeV each run, resulting in a range of 10 MeV to 10000 MeV(10 GeV). This energy range and spacing between runs was chosen in order to investigate:
     - If the detector is suitable for simulations over multiple(3) orders of magnitude
     - A reasonably dense set of points around our current simulation data at 100 MeV

In [ ]:
#Contains all of the simulated data for the simple detector
dataFolder = os.path.join(os.getcwd(), 'DataDetec1/')

columnNames = ['True', 'Argon1', 'Argon2', 'Argon3', 'Argon4', 'Argon5', 'Front', 'Back']

#The 5000 sample 100 MeV datasets for our three particles
eData = pd.read_csv(dataFolder + 'detec_1_electron_100.csv', comment = '#', header = None, names = columnNames)
pData = pd.read_csv(dataFolder + 'detec_1_gamma_100.csv', comment = '#', header = None, names = columnNames)
mData = pd.read_csv(dataFolder + 'detec_1_muon_100.csv', comment = '#', header = None, names = columnNames)

#Calibration data for the particles, 10 - 10000 MeV in 10MeV gaps
eCal = pd.read_csv(dataFolder + 'detec_1_electron_calibration.csv', comment = '#', header = None, names = columnNames)
pCal = pd.read_csv(dataFolder + 'detec_1_gamma_calibration.csv', comment = '#', header = None, names = columnNames)
mCal = pd.read_csv(dataFolder + 'detec_1_muon_calibration.csv', comment = '#', header = None, names = columnNames)

### Screening
 - In the simulations it is possible to detect 0 of a particles energy, for instance if its trajectory was not toward any of the sensitive plates. 
 - This causes mathematical problems down the line and doesn't add any extra information to our eventual classification task and so they can be removed here.

In [ ]:
#Screening the mian data files 
eScr = dataScreening(eData, 'e- 100 MeV')
pScr = dataScreening(pData, 'photon 100 MeV')
mScr = dataScreening(mData, 'mu+ 100 MeV')

#Screening the large energy range files 
eCalScr = dataScreening(eCal, 'e-')
pCalScr= dataScreening(pCal, 'photon')
mCalScr = dataScreening(mCal, 'mu+')

 - <font color='0B1FD1'>As can be seen here, the screening procedure found 3 counts of zero detected energy in the 100 MeV muon set of data and successfully removed them
</font>

### Calibrations
 - It is safe to assume that in the simulations, each fired particle may not necessarily have its true energy captured by our detecting layers. 
 - In order to account for this or any extra energy that is detected, we calibrate each set of data using the known truth energy.
 - Repeating this procedure over a much larger energy spectrum allows us to investigate the wider performance of our constructed detector. i.e can it be reliably used for simulations over some orders of magnitude or is it very sensitive to different energy ranges.

In [ ]:
#Adds additional data to each of our dataframes, Esum/ Ecal/ Calibrations

#Handles 100 MeV datasets
eAdd = calibration(eScr)
pAdd = calibration(pScr)
mAdd = calibration(mScr)

#Handles the large enegry range daatsets
eCalAdd = calibration(eCalScr)
pCalAdd = calibration(pCalScr)
mCalAdd = calibration(mCalScr)

In [ ]:
#Creates a custom subplot grid for our graphs to be displayed less cumbersomely, used thorughout 

fig, ax = plt.subplots(3, squeeze = False, sharex = True,  figsize=(7,7))

#Uses our utility function calibration1d which simply plots the 1d histograms
hist1 = calibration1d(eAdd, 'Electrons', 100, ax[0][0])
hist2 = calibration1d(pAdd, 'Photons', 100, ax[1][0])
hist3 = calibration1d(mAdd, 'Muons', 100, ax[2][0])
fig.tight_layout()
plt.show()

 - <font color='0B1FD1'>All of the calibration histograms follow a roughly gaussian distribution
</font>
 - <font color = '0B1FD1'>Both the electron and photon calibration graphs have significantly more spread and thus have a higher resolution value</font>
 - <font color = '0B1FD1'>The calibration graph for muons at 100 MeV is significantly less spread and is more centered around 0. This goes along with a much smaller resolution, about a third of the other two particles</font>
 - <font color = '0B1FD1'>This suggests that much more of the energy of the fired muons is accounted for</font>
 - <font color = '0B1FD1'>This could be due to the fact that muons are more penetrating that the other two particles and hence reflect less energy in other directions when hitting an object but at this energy are not penetrating enough to pass straight though the detecting structures</font>

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (12, 4))

#Uses out utility function calibration2d which simply plots the 2d histogram
hist1 = calibration2d(eCalAdd, 'Electrons', ax[0])
hist2 = calibration2d(pCalAdd, 'Photons', ax[1])
hist3 = calibration2d(mCalAdd, 'Muons', ax[2])
fig.tight_layout()
plt.show()

 - <font color='0B1FD1'>This data seems to paints a different picture to the one just seen with calibration measures for a single energy value </font>
 - <font color = '0B1FD1'>Both the electron and photon seem to maintain a very similar level of calibration/resolution after an initial period of larger spread. After ~ 2000-3000 MeV for both we see a sharp increase in the number calibration values close to zero</font>
 - <font color = '0B1FD1'>The same behaviour is not seen in the muon plot, where what is observed is a dropoff and then leveling out of calibration values. This is suggestive of some behaviour that should be investigated further.</font>
 - <font color = '0B1FD1'>Overall from the electron and photon plots, it would be reasonable to claim that the detector works at least between energies 10-10000 MeV, 3 orders of magnitude</font>

In [ ]:
plt.hist2d(mCalAdd['True'] , mCalAdd['Esum']/ mCalAdd['True'] , bins = (10, 20) )
plt.xlabel('True Eneergy [MeV]')
plt.ylabel('Esum / True')
plt.title('Ratios of Captured Energies for Muons')
plt.show()

 - <font color = '0B1FD1'>This new plot shows a different aspect to that of the calibration 2d histogram in the sense that we can investigate the muon's behaviour inside the detector more intuitively using  the ratio of energy accounted for vs the true energy</font>
 - <font color = '0B1FD1'>At lower true energy values, ~10-1000 MeV, it appears as though almost all of the fired muon's energy is accounted for in the detecting plates</font>
 - <font color = '0B1FD1'>However at higher energies, > ~1500 MeV, a lot of the energy of the muon seems to be 'lost', not detected by our sensitive detectors. This is likely due to the penetrating nature of higher energy muons</font>
 - <font color = '0B1FD1'>When calculating the calibrated values, we use these plotted ratios in an averaged sense. This explains why the lower energy spectrum of our previous histogram was so heavily skewed. The higher energy ratios differ largely from the lower energy ones and so when averaging these ratios and then applying that to all values, the low energy values end up getting mistreated quite significantly</font>
 - <font color = '0B1FD1'>Due to this, we can conclude that the detector does not work well for this range of energies for the muon particle</font>
 - <font color = '0B1FD1'>In the next sections we aim to classify these particles at 100 MeV and hence justify the use of muons here as the particle seems fairly 'well constrained' at E values around 10^1 - 10^3 MeV</font>

### Data Preprocessing and Visualisation
 - Add labels to the data
 - Compile into one large data set and shuffle
 - Create class balanced traning and testing splits

In [ ]:
#Adds typing to each dataset
eAdd['Type'] = 'electron'
pAdd['Type'] = 'photon'
mAdd['Type'] = 'muon'

#Creates one large dataframe with all data and then shuffles it
compiled = eAdd.append([pAdd, mAdd])
compiled = shuffle(compiled, random_state = 0)

In [ ]:
#g = sns.pairplot(data=compiled, vars=['Front', 'Argon1', 'Esum', 'Calibrations'], hue='Type', height=3)
g = plot.pair_grid(compiled, 'Type', features = ['Front', 'Argon1', 'Esum', 'Calibrations'])

 - <font color = '0B1FD1'>From the pair plot shown above we see that in general the muon is fairly apart from both the electron and photon and so we expect a classifier to have much more ease separating it out</font>
 - <font color = '0B1FD1'>Electrons and photons in general follow very similar trends and overlap in a lot of areas. Saying this, form the 'Front' variable(front shield) histogram that the particles clearly deposit a very different amount of energy here. Making this variable a likely contender to be one of the first splits something like a entropy based decision tree would use to try and tell electrons and photons apart</font>

In [ ]:
compiled.head(5)

In [ ]:
#Sets aside the class labels and the actual input data
y = compiled['Type'].values

#stores the data with no true values or labels in datframe format to later get column names
XDF = compiled.drop(['True', 'Type'], axis = 1)
X = XDF.values

#We pick a 80/20 train/test split thorughout the investigations for consistency, stratify is set to ensure balanced classes
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size=0.2, random_state=0, stratify = y)

for i in [X_train, X_test, y_train, y_test]:
    print('Shape: {}'.format(i.shape))

 - <font color = '0B1FD1'>The data has now been put into a suitable format for training and evaluation with a ML algorithm</font>

### Machine Learning
 - It was found last section that there does exist some variables in the dataset which could make it fairly easy for a simple algorithm to do well in the classification task. 
 - For this set of data we will classify using tree algorithms:
     - Simple decision tree, capped at 2 layers
     - An optimised decision tree
     - A random forest 

#### Simple Decision Tree
 - Throughout the notebook, all of the decision trees trained and used to predict are set up with the entropy criterium. This means that the splits made by the tree will be based on highest info gain out of the available variables. This is done in a greedy manner and so the 'best' decision tree created may not be the theoretical best, however searching for this configuration would be exponential in number of trees created.  

In [ ]:
dt = DecisionTreeClassifier(criterion='entropy', max_depth=2, random_state=0)
dt = dt.fit(X_train, y_train)

In [ ]:
#Using sklearn.ttree allow sus to plot the tree
plt.subplots(1, figsize = (15, 10))
tree.plot_tree(dt.fit(X_train, y_train), filled = True, class_names = np.unique(y_train), feature_names = XDF.columns) 
plt.show()

In [ ]:
#Predicts for both the training and testing datasets to get y labels which can be put into an accuracy function
predictTrain = dt.predict(X=X_train)
predictTest = dt.predict(X=X_test)
print('Classification accuracy on training set: {:.3f}'.format(accuracy_score(y_train,predictTrain)))
print('Classification accuracy on test set: {:.3f}'.format(accuracy_score(y_test,predictTest)))

 - <font color = '0B1FD1'>The testing accuracy is lower than the training, this is good as it follows what we generally would expect to happen with the classifier having actually been trained on the training set</font>
 - <font color = '0B1FD1'>For a simple 2 layer decision tree this is a very good accuracy, already correctly classifying ~95% of unseen cases</font>

#### Optimised Decision Tree

In [ ]:
trainAcc = []
testAcc = []

#Iterate over a series of depths, storing our training and testing results ecah time
depths = [1, 2, 4, 6, 8, 16]
for i in depths:
    dt = DecisionTreeClassifier(criterion='entropy', max_depth=i, random_state=0)
    dt = dt.fit(X_train, y_train)
    predictTrain = dt.predict(X=X_train)
    predictTest = dt.predict(X=X_test)
    trainAcc.append(accuracy_score(y_train,predictTrain))
    testAcc.append(accuracy_score(y_test,predictTest))

In [ ]:
#Finds the index of the max testing accuracy
maxTestIndex = np.argmax(testAcc)

fig, ax = plt.subplots(1, figsize = (15, 5))
ax.plot(depths, trainAcc)
ax.plot(depths, testAcc)
plt.axvline(depths[maxTestIndex], linestyle = '--', color = 'k')
plt.xlabel('Tree Depth')
plt.ylabel('Accuracy')
plt.title('Hyper Parameter optimisation of Single Decison Tree')
plt.show()

 - <font color = '0B1FD1'>At a depth of ~6, the decision tree starts to become over-fit and preform worse than previous on the testing set while the accuracy on the training set continues to improve</font>
 - <font color = '0B1FD1'>If allowed, the decision tree would run to a depth where every training instance was classified exactly before stropping and thus training accuracy would be 100%. This would be the maximum level of over-fitting possible and would likely result in poorer performance on the testing set </font>
 - <font color = '0B1FD1'>From this we choose depth = 6 as our optimal as it has the highest testing accuracy before diverging </font>

In [ ]:
chosenDepth = depths[maxTestIndex]
dt = DecisionTreeClassifier(criterion='entropy', max_depth=chosenDepth, random_state=0)
dt = dt.fit(X_train, y_train)
predictTrain = dt.predict(X=X_train)
predictTest = dt.predict(X=X_test)
print('Classification accuracy on training set using Depth = {}: {:.3f}'.format(chosenDepth, accuracy_score(y_train,predictTrain)))
print('Classification accuracy on test set using Depth = {}: {:.3f}'.format(chosenDepth, accuracy_score(y_test,predictTest)))

disp = confusion_matrix(y_test, predictTest)
print('\n')
print(disp)
print('Electron', 'Muon', 'Photon')


<font color = '0B1FD1'>Looking at this confusion matrix we can notice a couple of things:</font>
 - <font color = '0B1FD1'>The muons are perfectly classified with absolutely no overlap into either of the other two possible classes. This is most likely due to the fact, previously noted, that in some of the variable distributions, the muon set of data is very well separated form both the electrons and photons</font>
 - <font color = '0B1FD1'>The electrons and photons both have an overlap with one another but in general electrons are more often predicted for photons than photons predicted for electrons </font>

#### Random Forest
 - There are two hyper parameters for a random forest, n_estimators which is the number of trees created alongside the familiar max_depth. 
 - Due to this a proper hyper-parameter optimisation requires a grid search, this process is generally computationally expensive and wont be performed. Saying this, it is likely that doing so would likely reveal a combination of parameters that is able to perform marginally better than the base settings and so if the main aim was optimal performance, this would be a attractive option.
 - The parameters that will be used for now are the optimised depth from the simple decision tree, although not likely to be the optimal here, but potentially better than just a random designation,  and the default number of trees - 100.

In [ ]:
#Sets up a ranodm forest, a forest of decision trees 
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth = chosenDepth, random_state = 0)
rf = rf.fit(X_train, y_train)

predictTrain = rf.predict(X=X_train)
predictTest = rf.predict(X=X_test)

print('Classification accuracy on Training set: {:.4f}'.format(accuracy_score(y_train, predictTrain)))
print('Classification accuracy on test set: {:.4f}'.format(accuracy_score(y_test, predictTest)))

disp = confusion_matrix(y_test, predictTest)
print('\n')
print(disp)
print('Electron', 'Muon', 'Photon')

 - <font color = '0B1FD1'>A very similar trend is seen in the random forest's confusion matrix as was seen in the optimised decision tree's where muons have all been perfectly classified and there is only overlap between the electrons and photons</font>
 - <font color = '0B1FD1'>The main difference is the smaller number of times that an electron is predicted as a photon</font>
 - <font color = '0B1FD1'>In the testing set, the random forest performed about 0.1% better </font>

### Energy Range Of Classification Accuracy
 - So far the classification accuracy has been investigated at one energy value, 100 MeV. It would be useful to see how this accuracy varies over a selection of energies.
 - Since we have already discussed how the muon loses a lot of its energy at relatively high particle energies, it would be interesting to see if this actually effects the ability of a classifier in separating out the classes.
 - The energy range that will be investigated will be between 100 MeV and 3100 MeV with steps of 500 MeV. As this is a fairly sparse set of energy values, the datasets for each will be of a fair size of 2000 per particle per energy. These are smaller datasets than the ones used so far and thus have the potential to lower the accuracy of the classifier, although since we are investigating any potential trend, absolute differences are irrelevant.
 - As the decision tree is very close to the performance of the random forest and is quite a bit cheaper, it will be used.
 - There will be two different studies conducted here:
     - Firstly we will investigate how resolution varies as a function of true energy 
     - Secondly the energies will all be separated out in different training and testing sets each time, similar to what has been done with 100 MeV. When doing this we will also iterate over max depths with the aim to create accuracy heat maps. 

In [ ]:
#Data Imports
dataFolderMixed = os.path.join(os.getcwd(), 'DataDetec1/')
columnNames = ['True', 'Argon1', 'Argon2', 'Argon3', 'Argon4', 'Argon5', 'Front', 'Back']
eleMixed = pd.read_csv(dataFolderMixed + 'electron_mixed.csv', comment = '#', header = None, names = columnNames)
phoMixed = pd.read_csv(dataFolderMixed + 'gamma_mixed.csv', comment = '#', header = None, names = columnNames)
muMixed = pd.read_csv(dataFolderMixed + 'muon_mixed.csv', comment = '#', header = None, names = columnNames)

#Screens data for zero detected energy
eleScr = dataScreening(eleMixed, 'e- Mixed')
phoScr = dataScreening(phoMixed, 'photon Mixed')
muScr = dataScreening(muMixed, 'Mu+ Mixed')#

#Calibrates energy values
eleCal = calibration(eleScr)
phoCal = calibration(phoScr)
muCal = calibration(muScr)

In [ ]:
#First we look at how resolution varies over energy

#Sets up an energy array 
energies = np.arange(100, 3600, 500)

resEl = []
resPho = []
resMu = []
for i in range(len(energies)):
    #Creates new subset dataframes
    ele = eleCal[eleCal['True'] == energies[i]]
    pho = phoCal[phoCal['True'] == energies[i]]
    mu = muCal[muCal['True'] == energies[i]]

    resEl.append(np.std(ele['Calibrations']))
    resPho.append(np.std(pho['Calibrations']))
    resMu.append(np.std(mu['Calibrations']))

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (20, 5))
ax[0].plot(energies, resEl)
ax[1].plot(energies, resPho)
ax[2].plot(energies, resMu)

ax[0].set_xlabel('Energy MeV')
ax[1].set_xlabel('Energy MeV')
ax[2].set_xlabel('Energy MeV')

ax[0].set_ylabel('Energy Resolution')

ax[0].set_title('Electron Resolution vs Energy')
ax[1].set_title('Photon Resolution vs Energy')
ax[2].set_title('Muon Resolution vs Energy')

plt.show()

 - <font color =  '0B1FD1'>For all of the investigated particles, the resolution decreases quickly as a function of energy and eventually starts to level out, meaning the calibration gets tighter over time</font>
 - <font color =  '0B1FD1'>For muons, there is much sharper initial decrease in comparison to photons ans electrons. This is in agreement with what we seen previously, where at higher energies, muons have a very concentrated calibration value</font>

In [ ]:
#This large chunck of code preforms the DT fitting and evaluation for all of the energies we have data for at varying depths

energies = np.arange(100, 3600, 500)
depths = [2, 4, 6, 8, 16, 32, 64]

trainAcc = np.zeros((len(energies), len(depths)))
testAcc = np.zeros((len(energies), len(depths)))

for i in range(0, len(energies)):
    for k in range(0, len(depths)):
        ele = eleCal[eleCal['True'] == energies[i]]
        pho = phoCal[phoCal['True'] == energies[i]]
        mu = muCal[muCal['True'] == energies[i]]

        #Adds typing to each dataset
        ele['Type'] = 'electron'
        pho['Type'] = 'photon'
        mu['Type'] = 'muon'

        #Creates one large dataframe with all data and then shuffles it
        compiled = ele.append([pho, mu])
        compiled = shuffle(compiled, random_state = 0)

        y = compiled['Type'].values
        X = compiled.drop(['True', 'Type'], axis = 1).values

        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size=0.2, random_state=0, stratify = y)

        #Trains tree
        dt = DecisionTreeClassifier(criterion='entropy', max_depth=depths[k], random_state=0)
        dt = dt.fit(X_train, y_train)

        #uses trained tree to predict
        predictTrain = dt.predict(X=X_train)
        predictTest = dt.predict(X=X_test)
    
        test = accuracy_score(y_test,predictTest) 
        train = accuracy_score(y_train,predictTrain)
        
        trainAcc[i][k] = train
        testAcc[i][k] = test

#Rotates out arrays so we plot correctly, we rotate by 270 degrees
trainAcc = np.rot90(trainAcc, k=3, axes=(0, 1))
testAcc = np.rot90(testAcc, k=3, axes=(0, 1))

In [ ]:
#plots a color cooridinated grid for both our training and testing accuracies

fig, ax = plt.subplots(1, figsize = (8, 8))

im = plt.imshow(trainAcc)
ax.grid(color='k', linestyle='-', linewidth=0)
ax.set_xticks(np.arange(len(energies)))
ax.set_yticks(np.arange(len(depths)))
ax.set_xticklabels(energies)
ax.set_yticklabels(depths)
ax.set_xlabel('Energy MeV')
ax.set_ylabel('Depth of Tree')
for (j,i),label in np.ndenumerate(trainAcc):
    ax.text(i,j,round(label,2),ha='center',va='center', color = 'w')    
ax.set_title('Training Accuracies')
plt.colorbar()
plt.show()

fig, ax = plt.subplots(1, figsize = (8, 8))

im = plt.imshow(testAcc, aspect = 'equal')
ax.grid(color='k', linestyle='-', linewidth=0)
ax.set_xticks(np.arange(len(energies)))
ax.set_yticks(np.arange(len(depths)))
ax.set_xticklabels(energies)
ax.set_yticklabels(depths)
ax.set_title('Testing Accuracies')
ax.set_xlabel('Energy MeV')
ax.set_ylabel('Depth of Tree')
for (j,i),label in np.ndenumerate(testAcc):
    ax.text(i,j,round(label,2),ha='center',va='center', color = 'w')
plt.colorbar()
plt.show()

- <font color =  '0B1FD1'>The first thing to note is: Looking at the training plot, we see that at high number of layers, we tend toward and hit 100% accuracy. This agrees with what we expect in regards to over-fitting and is a nice confirmation that the fitting is working correctly</font>
- <font color =  '0B1FD1'>To Notice from Testing Plot:</font>
     - <font color =  '0B1FD1'>At higher depths, where we seen perfect training accuracy, test accuracy has fallen or stays constant, this is also in agreement with what we believe to be true for over-fitting</font>
     - <font color =  '0B1FD1'>At the highest energy investigated we seem to have better prediction accuracy, with these better accuracies at lower number of layers, ~1-6</font>
     - <font color =  '0B1FD1'>We have particularly poor performance at E = 2600 MeV</font>
     - <font color =  '0B1FD1'>In general, accuracy doesn't seem to follow any particular trend throughout the energy range, however does show more extreme behaviour toward higher energy</font>

# Improvements + Detector Two
 - In this section we aim to 'improve' our process from simulation to classification, this could cover a whole host of changes
 - The classification so far has mostly been done on a very specific set of particles. We then aim to make our classification perform 'well' for mixed particle simulation data this will be attempted by:
     - redesigning the detector to try to span into classifying hadrons
     - Generating new and mixed particle datasets
     - 'Cherry Picking' variables and feature engineering
     - Choosing algorithms more purposefully for our dataset
 - The new detector set up is based around the idea of both an electromagnetic and hadronic calorimeter. It is build with the following components/sections:
     - The 'EM Calorimeter':
         - Built from interwoven active silicon plates and absorbing lead plates
     - The 'Hadronic Calorimeter':
         - Built form interwoven active liquid argon plates and absorbing uranium plates
     - The whole detector is surrounded by a lead cylinder to capture particles moving away 
 - Obviously this is a full redesign of the detector and isn't really an 'improvement' as such. The actual improvement and comparison will come when we use feature engineering to try and get better classification accuracy.

### Import Data
 - As we are looking to investigate a larger range of particle types this time around, we will have to use much more mixed data at 1000 MeV:
     - For each of the following particles the dataset contains 5000 Data Points:
         - Electron
         - Positron
         - Photon
         - Pion 0
         - Kaon
         - Proton
         - Neutrons
 - Throughout this next section, since we are working with many more files etc, the code dealing with imports, screening and calibration becomes slightly more cumbersome but eases off toward the machine learning section

In [ ]:
#Sets up out two different data paths, first for main data, seconf for range of energy calibration data

dataFolder = os.path.join(os.getcwd(), 'ImprovData/')
dataFolderCal = os.path.join(os.getcwd(), 'ImprovData\\', 'cal\\')

columnNames = ['True', 'EM1', 'EM2', 'EM3', 'EM4', 'EM5', 'HAD1', 'HAD2', 'HAD3', 'HAD4', 'HAD5', 'HAD6',
              'HAD7', 'HAD8', 'HAD9', 'HAD10', 'HAD11', 'HAD12', 'HAD13', 'HAD14', 'HAD15', 'HAD16', 'Shield']

#The 5000 sample 1 GeV datasets for our seven Particles
elData = pd.read_csv(dataFolder + 'electron_1.csv', comment = '#', header = None, names = columnNames)
posData = pd.read_csv(dataFolder + 'positron_1.csv', comment = '#', header = None, names = columnNames)
phoData = pd.read_csv(dataFolder + 'gamma_1.csv', comment = '#', header = None, names = columnNames)
pionData = pd.read_csv(dataFolder + 'pion_1.csv', comment = '#', header = None, names = columnNames)
kaonData = pd.read_csv(dataFolder + 'kaon_1.csv', comment = '#', header = None, names = columnNames)
proData = pd.read_csv(dataFolder + 'proton_1.csv', comment = '#', header = None, names = columnNames)
neuData = pd.read_csv(dataFolder + 'neutron_1.csv', comment = '#', header = None, names = columnNames)

#The energy range sampled calibration data, 1-100.5 GeV with 0.5 GeV steps
elCal = pd.read_csv(dataFolderCal + 'e_cal.csv', comment = '#', header = None, names = columnNames)
posCal = pd.read_csv(dataFolderCal + 'pos_cal.csv', comment = '#', header = None, names = columnNames)
phoCal = pd.read_csv(dataFolderCal + 'gamma_cal.csv', comment = '#', header = None, names = columnNames)
pionCal = pd.read_csv(dataFolderCal + 'pi_cal.csv', comment = '#', header = None, names = columnNames)
kaonCal = pd.read_csv(dataFolderCal + 'kaon_cal.csv', comment = '#', header = None, names = columnNames)
proCal = pd.read_csv(dataFolderCal + 'proton_cal.csv', comment = '#', header = None, names = columnNames)
neuCal = pd.read_csv(dataFolderCal + 'neutrons_cal.csv', comment = '#', header = None, names = columnNames)

### Screening

In [ ]:
#Screening the mian data files 
eleData = dataScreening(elData, 'e- 1 GeV')
posData = dataScreening(posData, 'e+ 1 GeV')
phoData = dataScreening(phoData, 'Gamma 1 GeV')
pionData = dataScreening(pionData, 'pion 1 GeV')
kaonData = dataScreening(kaonData, 'kaon 1 GeV')
proData = dataScreening(proData, 'proton 1 GeV')
neuData = dataScreening(neuData, 'neutron 1 GeV')

#Screening Calibration Files
eleCal = dataScreening(elCal, 'e-')
posCal = dataScreening(posCal, 'e+')
phoCal = dataScreening(phoCal, 'Gamma')
pionCal = dataScreening(pionCal, 'pion')
kaonCal = dataScreening(kaonCal, 'kaon')
proCal = dataScreening(proCal, 'proton')
neuCal = dataScreening(neuCal, 'neutron')

- <font color =  '0B1FD1'>We find that kaons and neutrons give us the most 0 energy counts out of our particle types</font>
- <font color =  '0B1FD1'>These are our two heaviest particles(although neutron only by a little) and so maybe this is the connection</font>

### Calibration

In [ ]:
#Handles 1 GeV datasets

elAdd = calibration(elData)
posAdd = calibration(posData)
phoAdd = calibration(phoData)
pionAdd = calibration(pionData)
kaonAdd = calibration(kaonData)
proAdd = calibration(proData)
neuAdd = calibration(neuData)

#Handles the large enegry range daatsets
elAddCal = calibration(elCal)
posAddCal = calibration(posCal)
phoAddCal = calibration(phoCal)
pionAddCal = calibration(pionCal)
kaonAddCal = calibration(kaonCal)
proAddCal = calibration(proCal)
neuAddCal = calibration(neuCal)

In [ ]:
fig, ax = plt.subplots(7, squeeze = False,  figsize=(20,20), sharex = True)
hist1 = calibration1d(elAdd, 'Electrons', 1000, ax[0][0])
hist2 = calibration1d(posAdd, 'Positrons', 1000, ax[1][0])
hist3 = calibration1d(phoAdd, 'Photons', 1000, ax[2][0])
hist4 = calibration1d(pionAdd, 'Pions', 1000, ax[3][0])
hist5 = calibration1d(kaonAdd, 'Kaons', 1000, ax[4][0])
hist6 = calibration1d(proAdd, 'Protons', 1000, ax[5][0])
hist7 = calibration1d(neuAdd, 'Neutorns', 1000, ax[6][0])
fig.tight_layout()
plt.show()

- <font color =  '0B1FD1'>For out lighter/massless particles(electrons, positrons, photons and pions) we find a much narrower calibration graph, with correspondingly smaller resolutions</font>
- <font color =  '0B1FD1'>Our more massive particles(kaons, protons and neutrons, have a much more greater spread of calibration values, however still mostly centered on 0. These particles thus have a significantly larger resolution</font>
- <font color =  '0B1FD1'>Trying to reason this:</font>
    - <font color =  '0B1FD1'>As the heavier particles seemingly penetrate much further, there may be a higher possibility that they can escape the detector's shielding with sufficient energy left as to make the calibration values more varied. That being said, many don't and still allow a peak to form roughly around 0 </font>
    - <font color =  '0B1FD1'>In the case of the lighter/massless particles, after they enter the first calorimeter structure, they have a sufficiently low energy that they mostly cannot escape, losing detectable energy</font>

In [ ]:
fig, ax = plt.subplots(3, 3, figsize = (15, 15), squeeze = False, sharex = True)
hist1 = calibration2d(elAddCal, 'Electrons', ax[0][0])
hist2 = calibration2d(posAddCal, 'Positrons', ax[0][1])
hist3 = calibration2d(phoAddCal, 'Photons', ax[0][2])
hist4 = calibration2d(pionAddCal, 'Pions', ax[1][0])
hist5 = calibration2d(kaonAddCal, 'Kaons', ax[1][1])
hist6 = calibration2d(proAddCal, 'Protons', ax[1][2])
hist7 = calibration2d(neuAddCal, 'Neutrons', ax[2][0])

fig.tight_layout()
plt.show()

 - <font color =  '0B1FD1'>First row of particles(electrons, positrons and photons):</font>
    - <font color =  '0B1FD1'>All of these particles show a very similar trend and one we have seen before, where we start off with more spread with that eventually tightening up at higher energy values</font>
    - <font color =  '0B1FD1'>In general however, all of these particles have a fairly low spread as can be seen by the y-axis range</font>
 - <font color =  '0B1FD1'>Second Set of particles(pions, kaons and protons):</font>
    - <font color =  '0B1FD1'>For all of these particles the y-axis range is much larger than the previous range(factor of 3 approximately), this signifies the larger calibration range that these particles occupy over a large energy range. Saying this the pion spread is still more in line with the previous row, confirming that the trend we see in the cell above with the 1d histogram is followed throughout varying energies </font>
    - <font color =  '0B1FD1'>For both kaons and protons, the actual spread is larger and the calibration seems to center on ~0.2 rather than 0</font>
 - <font color =  '0B1FD1'>Third Row(neutrons):</font>
    - <font color =  '0B1FD1'>Neutrons follow a very similar trend to protons and kaons with larger spread and centres off 0</font>
 - <font color =  '0B1FD1'>General:</font>
    - <font color =  '0B1FD1'>We generally see a confirmation of previous results where our lighter/massless particles have a much lower spread and typically center their calibrations about 0 and where out more massive particles have a shifted peak and larger spread</font>
    - <font color =  '0B1FD1'>In general we see fairly consistent calibrations across the full energy spectrum investigated for energy particle, based off of this, it would be reasonable to state that the detector set up works sufficiently well from between 1 - 100GeV at least. Ideally based on the construction created, it should be suitable for energies much larger, but simulation time starts to increase massively past 100 GeV seemingly so that is why only data in the range 1 - 100 GeV is collected </font>

### Data Preprocessing and Visualisation

In [ ]:
#Adds typing to each dataset
elAdd['Type'] = 'electron'
posAdd['Type'] = 'positron'
phoAdd['Type'] = 'photon'
pionAdd['Type'] = 'pion'
kaonAdd['Type'] = 'kaon'
proAdd['Type'] = 'proton'
neuAdd['Type'] = 'neutron'

#Creates one large dataframe with all data and then shuffles it
compiled = elAdd.append([posAdd, phoAdd, pionAdd, kaonAdd, proAdd, neuAdd])
compiled = shuffle(compiled, random_state = 0)

In [ ]:
#g = sns.pairplot(data=compiled, vars=['EM1', 'HAD3', 'Esum', 'Calibrations'], hue='Type', height=3)
g = plot.pair_grid(compiled, 'Type', features = ['EM1', 'HAD3', 'Esum', 'Calibrations'])

 - <font color =  '0B1FD1'>In most of the pair plots shown, we see mostly overlapping data with some different particles in each separating themselves in some manner</font>
 - <font color =  '0B1FD1'>Note that this is a very small sample of the number of variables in the dataset but visualising all of them would be very very cumbersome, an ~20x20 grid</font>
 - <font color =  '0B1FD1'>In both Esum and EM1 we see some separation in the data which may be lead these variables to be chosen above others</font>

In [ ]:
#Sets aside the class labels and the actual input data
y = compiled['Type'].values
XDF = compiled.drop(['True', 'Type'], axis = 1)
X = XDF.values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size=0.2, random_state=0, stratify = y)

for i in [X_train, X_test, y_train, y_test]:
    print('Shape: {}'.format(i.shape))

### Machine Learning On Raw Data
 - Attempting machine learning on the raw, however now calibrated, data is our way here of defining a baseline performance for what can be expected with this detector design and these particle types before attempting some feature engineering in an attempt to improve performance
 - The algorithms baselined here will include:
     - Optimised Decision Tree
     - K-Nearest Neighbors at K = 2(although will not be replicated later on)
     - Optimised KNN
     - Logistic Regression

#### Optimised Decision Tree

In [ ]:
trainAcc = []
testAcc = []

#As the tree is cheap, we use a wider and more dense selection of points
depths = np.arange(1, 71, 5)
for i in depths:
    print('Calculating for Depth = {}'.format(i))
    dt = DecisionTreeClassifier(criterion='entropy', max_depth=i, random_state=0)
    dt = dt.fit(X_train, y_train)
    predictTrain = dt.predict(X=X_train)
    predictTest = dt.predict(X=X_test)
    trainAcc.append(accuracy_score(y_train,predictTrain))
    testAcc.append(accuracy_score(y_test,predictTest))

In [ ]:
maxTestIndex = np.argmax(testAcc)

fig, ax = plt.subplots(1, figsize = (15, 5))
ax.plot(depths, trainAcc)
ax.plot(depths, testAcc)
plt.axvline(depths[maxTestIndex], linestyle = '--', color = 'k')
plt.xlabel('Tree Depth')
plt.ylabel('Accuracy')
plt.title('Hyper Parameter optimisation of Single Decison Tree')
plt.show()

 - <font color =  '0B1FD1'>From this plot we see that out of the sampled values for tree depth, 51 performed the best in our testing set, then-after levelling out and seemingly running parallel to training accuracy of 100%</font>
 - <font color =  '0B1FD1'>This is then the depth we perform a final base fit for</font>

In [ ]:
chosenDepth = depths[maxTestIndex]
dt = DecisionTreeClassifier(criterion='entropy', max_depth=chosenDepth, random_state=0)
dt = dt.fit(X_train, y_train)
predictTrain = dt.predict(X=X_train)
predictTest = dt.predict(X=X_test)
print('Classification accuracy on training set using Depth = {}: {:.3f}'.format(chosenDepth, accuracy_score(y_train,predictTrain)))
print('Classification accuracy on test set using Depth = {}: {:.3f}'.format(chosenDepth, accuracy_score(y_test,predictTest)))

disp = confusion_matrix(y_test, predictTest)
print('\n')
print(disp)
print('e-', 'e+', 'photon', 'pi0', 'kaon0', 'p', 'n')

 - <font color =  '0B1FD1'>An optimised decision tree performs very well on the mixed set of data, correctly classifying ~97% of unseen data. This is impressive given the number of classes to choose form </font>
 - <font color =  '0B1FD1'>Looking at the confusion matrix we see:</font>
      - <font color =  '0B1FD1'>There are no particular classes which do very badly. The worst predicted particle is protons which seem to be predicted as electrons fairly frequently</font>

#### KNN with K = 2
 - The KNN algorithm works by comparing new testing instances to n nearest neighbours, defined by some metric, in the training set

In [ ]:
nn = KNeighborsClassifier(n_neighbors=2)
nn = nn.fit(X_train, y_train)

predictTrain = nn.predict(X=X_train)
predictTest = nn.predict(X=X_test)

print('Classification accuracy on Training set: {:.4f}'.format(accuracy_score(y_train, predictTrain)))
print('Classification accuracy on test set: {:.4f}'.format(accuracy_score(y_test, predictTest)))

- <font color =  '0B1FD1'>This is just a trial of this algorithm just to get a feel for how it performs</font>

#### Optimising KNN
 - The KNN algorithm does not have a conventional training period whereafter it can be used to predict more quickly. Instead KNNs compare each point in the training set to each unseen data point and makes a decision bases off of the NNs class labels and some distance metric. For this reason the algorithm scales in time with the size of both the training and testing datasets, where the larger the dataset, the slower the full computation process. This makes KNN fairly slow in predictions in comparison to other methods which have most of their computational time in the training.
 - Speedups can be seen using variations such as KDtrees for low dimensional numeric datasets and Inverted lists for sparse catagorical data

In [ ]:
testAcc = []
trainAcc = []

#List of trial num of NNs to use
NNs = [1, 2, 4, 6, 8, 10, 12, 14, 16]

for k in NNs:
    print('Calculating for {} NNs'.format(k))
    nn = KNeighborsClassifier(n_neighbors = k)
    nn = nn.fit(X_train, y_train)

    predictTrain = nn.predict(X=X_train)
    predictTest = nn.predict(X=X_test)

    testAcc.append(accuracy_score(y_test, predictTest))
    trainAcc.append(accuracy_score(y_train, predictTrain))

In [ ]:
maxTestIndex = np.argmax(testAcc)

fig, ax = plt.subplots(1, figsize = (15, 5))
ax.plot(NNs, testAcc)
ax.plot(NNs, trainAcc)
plt.axvline(NNs[maxTestIndex], linestyle = '--', color = 'k')
plt.legend(['Test', 'Train'])
plt.xlabel('Number Nerest Neighbours')
plt.xticks(NNs)
plt.ylabel('Accuracy')
plt.title('Accuracy as Function of NNs')
plt.show()

 - <font color =  '0B1FD1'>From the plot wee that our best performing number of nearest neighbours is 10</font>
 - <font color =  '0B1FD1'>In comparison notice that:</font>
      - <font color =  '0B1FD1'>The training accuracy is 100% at NN =1, this is because if we only compare to the single nearest point, when comparing the dataset which the algorithm was trained on, its nearest point is always itself and so the class label is always predicted correctly</font>
       - <font color =  '0B1FD1'>Training accuracy tends downward rather than most other algorithms encountered where the training data becomes overfit </font>
 - <font color =  '0B1FD1'>We will now use the optimal Number of NNs to train our final model </font>

In [ ]:
maxTest = testAcc[maxTestIndex]
maxTrain = trainAcc[maxTestIndex]

print('The best test accuracy was found at NNs = {}'.format(NNs[maxTestIndex]))
print('Train: {:.2f} %'.format(maxTrain * 100))
print('Test: {:.2f} %'.format(maxTest * 100))

 - <font color =  '0B1FD1'>We see quite a poor performance in comparison to our optimised decision tree of only ~78.5% on the test set</font>

#### Logistic Regression
 - Although named as regression, Logistic Regression is actually a classification task which uses a base linear model and the logistic function to predict classes.
 - As the basis of the model is linear, logistic regression often performs well with linearly separable data. This is more similar to decision tree where axis aligned cuts are performed than KNN where only close-by points matter in the classification.
 - Unlike decision trees, logR is not limited to axis aligned cuts and can perform diagonal separations of data.

In [ ]:
#Sets up a logistic regression classifier
logR = LogisticRegression(solver = 'lbfgs', random_state=0, max_iter = 500, multi_class = 'auto')
logR = logR.fit(X_train, y_train)

predictTrain = nn.predict(X=X_train)
predictTest = nn.predict(X=X_test)

print('Classification accuracy on Training set: {:.4f}'.format(accuracy_score(y_train, predictTrain)))
print('Classification accuracy on test set: {:.4f}'.format(accuracy_score(y_test, predictTest)))

- <font color =  '0B1FD1'>We see a similar performance between logistic regression and KNN, both of which are significantly lower  than that of the optimised decision tree</font>

- <font color =  '0B1FD1'>Based off of how KNN and LogR function, it will be interesting to see if any significant improvement is seen after our feature engineering</font>

### Feature Engineering
 - Will now try to use feature engineering to improve our classification results from the last section
 - What will be attempted:
     - sum over both the em and hadronic calorimeters and represent the energy deposited by a particle into each as two variables, the first an energy sum and a second, the ratio of each of these over the detected energy

In [ ]:
#Sets how mnay layers/ columns we have of each calorimeter
nLayersEM = 5
nLayersHAD = 16

#Starts a df to append into
had = compiled['HAD1']
#Starts a lits to store column names, later to be dropped 
hadStrings = ['HAD1']
#Iterates thorigh the potential column names and adds all the data to one variable/series 
for i in range(1, nLayersHAD):
    string = 'HAD' + str(i+1)
    had = had +  compiled[string]
    #also appends actual column names to drop later 
    hadStrings.append(string)
    
#Follows same structure as previous HAD looping section
em = compiled['EM1']
emStrings = ['EM1']
for i in range(1, nLayersEM):
    string = 'EM' + str(i+1)
    em = em +  compiled[string]
    emStrings.append(string)
    
#Defines some new variables in the compiled dataframes, the sums over each type of calorimiter
compiled['HAD'] = had
compiled['EM'] = em

#Defines a new dataframe with the old split calorimter layer columns dropped in favour of our new variables 
compiledFeat = compiled.drop(hadStrings, axis = 1)
compiledFeat = compiledFeat.drop(emStrings, axis = 1)

#TYhe ratio variables
compiledFeat['Ratio EM'] = compiledFeat['EM']/compiledFeat['Esum']
compiledFeat['Ratio HAD'] = compiledFeat['HAD']/compiledFeat['Esum']

In [ ]:
#g = sns.pairplot(data=compiledFeat, vars=['EM', 'HAD', 'Ratio EM', 'Ratio HAD'], hue='Type', height=3)
g = plot.pair_grid(compiledFeat, 'Type', features = ['EM', 'HAD', 'Ratio EM', 'Ratio HAD'])

- <font color =  '0B1FD1'>Compared to the original pairplot of this data, our new variables seem to induce a much more visible splitting in the particle types</font>
- <font color =  '0B1FD1'>This has the potential to allow our algorithms to more easily separate out our classes</font>

In [ ]:
#Sets aside the class labels and the actual input data
y = compiledFeat['Type'].values
XDF = compiledFeat.drop(['True', 'Type'], axis = 1)
X = XDF.values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size=0.2, random_state=0, stratify = y)

for i in [X_train, X_test, y_train, y_test]:
    print('Shape: {}'.format(i.shape))

### Machine Learning on Feature Engineered Data
 - We will run the same algorithms as was ran on the raw data set for a truer comparison
 - All of the processes are identical outside of maybe optimal values found and so there are little accompanying comments as they can be found in the previous section

#### Optimised Decision Tree

In [ ]:
trainAcc = []
testAcc = []

depths = np.arange(1, 71, 5)
for i in depths:
    dt = DecisionTreeClassifier(criterion='entropy', max_depth=i, random_state=0)
    dt = dt.fit(X_train, y_train)
    predictTrain = dt.predict(X=X_train)
    predictTest = dt.predict(X=X_test)
    trainAcc.append(accuracy_score(y_train,predictTrain))
    testAcc.append(accuracy_score(y_test,predictTest))

In [ ]:
maxTestIndex = np.argmax(testAcc)

fig, ax = plt.subplots(1, figsize = (15, 5))
ax.plot(depths, trainAcc)
ax.plot(depths, testAcc)
plt.axvline(depths[maxTestIndex], linestyle = '--', color = 'k')
plt.xlabel('Tree Depth')
plt.ylabel('Accuracy')
plt.title('Hyper Parameter optimisation of Single Decison Tree')
plt.show()

In [ ]:
chosenDepth = depths[maxTestIndex]
dt = DecisionTreeClassifier(criterion='entropy', max_depth=chosenDepth, random_state=0)
dt = dt.fit(X_train, y_train)
predictTrain = dt.predict(X=X_train)
predictTest = dt.predict(X=X_test)
print('Classification accuracy on training set using Depth = {}: {:.3f}'.format(chosenDepth, accuracy_score(y_train,predictTrain)))
print('Classification accuracy on test set using Depth = {}: {:.3f}'.format(chosenDepth, accuracy_score(y_test,predictTest)))

disp = confusion_matrix(y_test, predictTest)
print('\n')
print(disp)
print('e-', 'e+', 'photon', 'pi0', 'kaon0', 'p', 'n')

 - <font color =  '0B1FD1'>Using our feature engineered data we see a small improvement of ~1% using a decision tree</font>
 - <font color =  '0B1FD1'>This is not particuarly substantial in and of itself however noting that we were already at 97%, a jump to 98% seems more impressive</font>
 - <font color =  '0B1FD1'>We also see that the optimal depth stays the same between the two different datasets</font>

#### Optimising KNN

In [ ]:
testAcc = []
trainAcc = []
NNs = [1, 2, 4, 6, 8, 10, 12, 14, 16]

for k in NNs:
    print('Calculating for {} NNs'.format(k))
    nn = KNeighborsClassifier(n_neighbors = k)
    nn = nn.fit(X_train, y_train)

    predictTrain = nn.predict(X=X_train)
    predictTest = nn.predict(X=X_test)

    testAcc.append(accuracy_score(y_test, predictTest))
    trainAcc.append(accuracy_score(y_train, predictTrain))

In [ ]:
maxTestIndex = np.argmax(testAcc)

fig, ax = plt.subplots(1, figsize = (15, 5))
ax.plot(NNs, testAcc)
ax.plot(NNs, trainAcc)
plt.axvline(NNs[maxTestIndex], linestyle = '--', color = 'k')
plt.legend(['Test', 'Train'])
plt.xlabel('Number Nerest Neighbours')
plt.xticks(NNs)
plt.ylabel('Accuracy')
plt.title('Accuracy as Function of NNs')
plt.show()

In [ ]:
maxTest = testAcc[maxTestIndex]
maxTrain = trainAcc[maxTestIndex]

print('The best test accuracy was found at NNs = {}'.format(NNs[maxTestIndex]))
print('Train: {:.2f} %'.format(maxTrain * 100))
print('Test: {:.2f} %'.format(maxTest * 100))

 - <font color =  '0B1FD1'>For out KNN algorithm we see a much more substantial improvement of ~8% with our new dataset</font>
 - <font color =  '0B1FD1'>We also find that the optimal number of NN is vastly different, and instead of 10 as it was with the OG features, it is now 1</font>

#### Logistic Regression

In [ ]:
logR = LogisticRegression(solver = 'lbfgs', random_state=0, max_iter = 400, multi_class = 'auto')
logR = logR.fit(X_train, y_train)

predictTrain = nn.predict(X=X_train)
predictTest = nn.predict(X=X_test)

print('Classification accuracy on Training set: {:.4f}'.format(accuracy_score(y_train, predictTrain)))
print('Classification accuracy on test set: {:.4f}'.format(accuracy_score(y_test, predictTest)))

 - <font color =  '0B1FD1'>We see a very similar improvement for our logistic regression algorithm as we seen in KNN</font>

### Energy Range Of Classification Accuracy
 - In order to investigate this fairly, we will use the same algorithm for each energy and only vary the parameters.
 - We have chosen the decision tree once again as it is fairly cheap and in this case performed the best out of our trialed algorithms

In [ ]:
#This full cell delas with reading in all of the datafiles for all particles at different energies and creating relevant dfs

mypath = os.path.join(os.getcwd(),'improvData\\', 'mixedData\\')
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

columnNames = ['True', 'EM1', 'EM2', 'EM3', 'EM4', 'EM5', 'HAD1', 'HAD2', 'HAD3', 'HAD4', 'HAD5', 'HAD6',
              'HAD7', 'HAD8', 'HAD9', 'HAD10', 'HAD11', 'HAD12', 'HAD13', 'HAD14', 'HAD15', 'HAD16', 'Shield']

el_files, pos_files, pho_files, pion_files,  = [], [], [], []
pro_files, neu_files, kaon_files = [], [], []

fileLists = [el_files, pos_files, pho_files, pion_files, kaon_files, pro_files, neu_files]

for i in onlyfiles:
    if 'el' in i:
        el_files.append(i)
    if 'pos' in i:
        pos_files.append(i)
    if 'gam' in i:
        pho_files.append(i)
    if 'pi' in i:
        pion_files.append(i)
    if 'kaon' in i:
        kaon_files.append(i)
    if 'pro' in i:
        pro_files.append(i)
    if 'neu' in i:
        neu_files.append(i)

elDf = pd.DataFrame(columns = columnNames)
posDf = pd.DataFrame(columns = columnNames)
phoDf = pd.DataFrame(columns = columnNames)
pionDf = pd.DataFrame(columns = columnNames)
kaonDf = pd.DataFrame(columns = columnNames)
proDf = pd.DataFrame(columns = columnNames)
neuDf = pd.DataFrame(columns = columnNames)

dfList = [elDf, posDf, phoDf, pionDf, kaonDf, proDf, neuDf]
newdf = []
for j in range(len(dfList)):
    df = dfList[j]
    for i in range(len(fileLists[j])):
        newDf = pd.read_csv(mypath + fileLists[j][i], comment = '#', header = None, names = columnNames)
        df = df.append([newDf])
    newdf.append(df)
    
elDf = newdf[0]
posDf =newdf[1]
phoDf = newdf[2]
pionDf = newdf[3]
kaonDf = newdf[4]
proDf = newdf[5]
neuDf = newdf[6]

In [ ]:
eleCal = dataScreening(elDf, 'e-')
posCal = dataScreening(posDf, 'e+')
phoCal = dataScreening(phoDf, 'Gamma')
pionCal = dataScreening(pionDf, 'pion')
kaonCal = dataScreening(kaonDf, 'kaon')
proCal = dataScreening(proDf, 'proton')
neuCal = dataScreening(neuDf, 'neutron')

elDf = calibration(eleCal)
posDf = calibration(posCal)
phoDf = calibration(phoCal)
pionDf = calibration(pionCal)
kaonDf = calibration(kaonCal)
proDf = calibration(proCal)
neuDf = calibration(neuCal)

 - <font color =  '0B1FD1'>We see again that non physical points often appear for both neutrons and kaons</font> 

In [ ]:
#Sets up our iterable parameters
#Very similar to previous section where we investigaed accurayc over energy

energies = np.arange(1000, 11000, 2000)
depths = [ 2, 4, 8, 16, 32, 64, 128]

trainAcc = np.zeros((len(energies), len(depths)))
testAcc = np.zeros((len(energies), len(depths)))

for i in range(0, len(energies)):
    for k in range(0, len(depths)):
        ele = elDf[elDf['True'] == energies[i]]
        pos = posDf[posDf['True'] == energies[i]]
        pho = phoDf[phoDf['True'] == energies[i]]
        pion = pionDf[pionDf['True'] == energies[i]]
        kaon = kaonDf[kaonDf['True'] == energies[i]]
        pro = proDf[proDf['True'] == energies[i]]
        neu = neuDf[neuDf['True'] == energies[i]]
        
        #Adds typing to each dataset
        ele['Type'] = 'electron'
        pos['Type'] = 'positron'
        pho['Type'] = 'photon'
        pion['Type'] = 'pion'
        kaon['Type'] = 'kaon'
        pro['Type'] = 'proton'
        neu['Type'] = 'neutron'

        #Creates one large dataframe with all data and then shuffles it
        compiled = ele.append([pos, pho, pion, kaon, pro, neu])
        compiled = shuffle(compiled, random_state = 0)

        #Performs the feature engineering
        nLayersEM = 5
        nLayersHAD = 16
        had = compiled['HAD1']
        hadStrings = ['HAD1']
        for l in range(1, nLayersHAD):
            string = 'HAD' + str(l+1)
            had = had +  compiled[string]
            hadStrings.append(string)
        em = compiled['EM1']
        emStrings = ['EM1']
        for m in range(1, nLayersEM):
            string = 'EM' + str(m+1)
            em = em +  compiled[string]
            emStrings.append(string)
        compiled['HAD'] = had
        compiled['EM'] = em
        compiledFeat = compiled.drop(hadStrings, axis = 1)
        compiledFeat = compiledFeat.drop(emStrings, axis = 1)
        compiledFeat['Ratio EM'] = compiledFeat['EM']/compiledFeat['Esum']
        compiledFeat['Ratio HAD'] = compiledFeat['HAD']/compiledFeat['Esum']
        
        y = compiledFeat['Type'].values
        X = compiledFeat.drop(['True', 'Type'], axis = 1).values

        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size=0.2, random_state=0, stratify = y)

        #Trains tree
        dt = DecisionTreeClassifier(criterion='entropy', max_depth=depths[k], random_state=0)
        dt = dt.fit(X_train, y_train)

        #uses trained tree to predict
        predictTrain = dt.predict(X=X_train)
        predictTest = dt.predict(X=X_test)
    
        test = accuracy_score(y_test,predictTest) 
        train = accuracy_score(y_train,predictTrain)
        
        trainAcc[i][k] = train
        testAcc[i][k] = test
        
trainAcc = np.rot90(trainAcc, k=3, axes=(0, 1))
testAcc = np.rot90(testAcc, k=3, axes=(0, 1))


In [ ]:
fig, ax = plt.subplots(1, figsize = (8, 8))

im = plt.imshow(trainAcc)
ax.grid(color='k', linestyle='-', linewidth=0)
ax.set_xticks(np.arange(len(energies)))
ax.set_yticks(np.arange(len(depths)))
ax.set_xticklabels(energies)
ax.set_yticklabels(depths)
ax.set_title('Training Accuracies')
ax.set_xlabel('Energy MeV')
ax.set_ylabel('Depth of Tree')
plt.colorbar()
for (j,i),label in np.ndenumerate(trainAcc):
    ax.text(i,j,round(label,2),ha='center',va='center', color = 'w')
plt.show()

fig, ax = plt.subplots(1, figsize = (8, 8))

im = plt.imshow(testAcc)
ax.grid(color='k', linestyle='-', linewidth=0)
ax.set_xticks(np.arange(len(energies)))
ax.set_yticks(np.arange(len(depths)))
ax.set_xticklabels(energies)
ax.set_yticklabels(depths)
ax.set_title('Testing Accuracies')
ax.set_xlabel('Energy MeV')
ax.set_ylabel('Depth of Tree')
plt.colorbar()
for (j,i),label in np.ndenumerate(testAcc):
    ax.text(i,j,round(label,2),ha='center',va='center', color = 'w')
plt.show()

 - <font color =  '0B1FD1'>Simply looking at these plots without using the colour bar as reference for magnitude, both look very similar where we start form low accuracy and increase as we go up in number of layers</font>
 - <font color =  '0B1FD1'>After ~64 layers, the accuracy for each energy stays essentially constant, changing by miniscule amounts, where a dividing line is barely visible between boxes</font>
 - <font color =  '0B1FD1'>in general the test plot shows there is little to no difference in accuracy as a function of energy form 1-9 GeV outside of maybe how quickly accuracy tends to its maximum, where this happens very slightly faster at lower energies.</font>

# Conclusions

 - <font color =  '0B1FD1'>Detector One: </font>
      - <font color =  '0B1FD1'>In the first detector design we attempted to classify electrons, photons and muons. It was found that muons, at higher energies behaved very differently to those at the energy we were primarily investigating. As they seemed to be fairly well constrained, and in fact having thr smallest energy resolution out of the three particles, at 100 MeV, we justified that classification was reasonable. We also justified that the detector could work over at least two orders of magnitude for electrons and photons.</font>
      - <font color =  '0B1FD1'>The bets performing classifier we trained on this data was the random forest which managed an accuracy of ~95.5% on unseen data. It was marginally better than the decision tree.</font>
      - <font color =  '0B1FD1'>We chose to use the decision tree for the multi energy study where we investigated accuracy as a function of energy and tree depth as it approached the accuracy of the random forest and was significantly cheaper per run. In this study there was no staggeringly clear trend found however it appeared as though higher energies were classified better than low energies are were done so at fairly low number of tree layers, ~1-4.</font>

 - <font color =  '0B1FD1'>Second Detector Design: </font>
      - <font color =  '0B1FD1'>From the first detector which was largely used for smaller lighter/massless particle, a new detector was constructed which was designed to work for these particles plus heavier species such as pion, kaons, protons and neutrons.the majority of simulation data for this detector was collected at an energy of 1 GeV</font>
      - <font color =  '0B1FD1'>The single energy calibration histogram showed that for lighter/massless species, the resolution was much smaller(calibration plot is tighter with a lower standard deviation) than what was observed for more massive particles. An attempt was made to reason this out physically in that section</font>
      - <font color =  '0B1FD1'>The 2d histograms spanning energies from 1-100 GeV showed that our detector was fairly well constructed and the resolution/calibration followed a similar trend throughout, covering 2 orders of magnitude</font>
      - <font color =  '0B1FD1'>A number of algorithms were tested with this 1 GeV dataset, including KNN, Logistic Regression and a Decision Tree. the decision tree performed best with an accuracy of ~97% with both of the other algorithms falling below 80%</font>
      
 - <font color =  '0B1FD1'>Feature Engineering and Improvement: </font>
      - <font color =  '0B1FD1'>The main improvement of this detector and classification simulation came in the form of some feature engineering, where new variables were created out of the existing data to try and improve classification accuracy. The 4 new variables created revolved around the energy deposited in each part of the detector, the EM and HAD calorimeter</font>
      - <font color =  '0B1FD1'>These new features boosted classification accuracy across the board for the algorithms previously used for this data. With a 1% increase in the decision tree and ~8% increase in both KNN and Logistic Regression</font>
      - <font color =  '0B1FD1'>The final study conducted was how the prediction accuracy varied as a function of energy for our second detector. It was observed that all energies behaved very similarly in this regard and no trend distinguishing them was found. In this study, the new features were used for every energy value's dataset as to make the performance comparable across energies.</font>